In [42]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet
from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim
from nltk.wsd import lesk
import string

In [43]:
#stop word removal
from nltk.corpus import stopwords
stop={'a', 'of', 'on', 'the'} #add related stop word here
print(stop)

{'a', 'of', 'on', 'the'}


In [44]:
df_csv= pd.read_csv('ask_fm_data.csv')

In [45]:
df_csv.head(10100)

,Unnamed: 0,q_a,label
0,0,how to tell if a guy is gay if they seem super...,0
1,1,Idk I'm not gay,0
2,2,This person is 100% fake so beware Isabella!,1
3,3,Oof lmao how is she fake? Who's this lmao shit...,1
4,4,If your loved one left n cut all ties w you sh...,0
...,...,...,...
9993,9993,Basically anything omg,0
9994,9994,Why is it that when I ask someone so random as...,0
9995,9995,Thats when my bitch mode comes out,0
9996,9996,Post a picture of your favorite celebrity!,0


In [46]:
#marge all data from files that has been read
q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(int(df_csv['label'][i]))
    q_a.append(df_csv['q_a'][i])
print(len(q_a))
print(len(label))

9998
9998


In [64]:
# text cleaning
w=list(string.ascii_lowercase)
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=sen.replace("@", '')
    sen=sen.replace("pu**y", 'pussy')
    sen=sen.replace(" mofo ", ' motherfucker')
    sen=sen.replace("NEWLINE_TOKEN", " ")
    sen=sen.replace("TAB_TOKEN", " ")
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("?", ' ? ')
    sen=sen.replace("!", ' ! ')
    sen=sen.replace(" y ", ' why ')
    sen=sen.replace(" u ", ' you ')
    sen=sen.replace(" w ", ' with ')
    sen=sen.replace(" mf ", ' motherfucker ')
    sen=sen.replace(" IDGAF ", ' i dont give a fuck ')
    sen=sen.replace(" IDRC ", ' I Dont Really Care ')
    sen=sen.replace(" tf ", ' the fuck ')
    sen=sen.replace("wtf ", 'what the fuck ')
    sen=sen.replace(" yanno ", ' you know ')
    sen=sen.replace(" igt ", ' i know right ')
    sen=sen.replace(" r ", ' are ')
    sen=sen.replace(" tbh ", ' to be honest ')
    sen=sen.replace(" ik ", ' i know ')
    sen=sen.replace(" af ", ' as fuck ')
    sen=sen.replace(" hes ", ' he is ')
    sen=sen.replace(" des ", ' this ')
    sen=sen.replace(" bout ", ' about ')
    sen=sen.replace(" em ", ' them ')
    sen=sen.replace(" stg ", ' swear to god ')
    sen=sen.replace(" bj ", ' blow job ')
    sen=sen.replace(" ig ", ' i guess ')
    sen=sen.replace(" fagg ", ' faggot ')
    sen=sen.replace(" fag ", ' faggot ')
    sen=sen.replace(" fagot ", ' faggot ')
    sen=sen.replace(" tfw ", ' That Feel When ')
    sen=sen.replace(" wa ", ' was ')
    sen=sen.replace(" n ", ' and ')
    sen=sen.replace(" y0 ", ' you ')
    sen=sen.replace(" dat ", ' that ')
    sen=sen.replace(" yo ", ' you ')
    sen=sen.replace("youre ", ' you are ')
    sen=sen.replace(" v ", ' vagina ')
    sen=sen.replace(" pusy ", ' pussy ')
    sen=sen.replace(" pusy", ' pussy ')
    sen=sen.replace("pussie", 'pussy')
    sen=sen.replace(" v.", ' vagina ')
    sen=sen.replace(" doggystyle ", ' doggy style ')
    sen=sen.replace("di ck", 'dick')
    sen=sen.replace("r you", 'are you')
    sen=sen.replace("r u", 'are you')
    sen=sen.replace("f*ck", 'fuck')
    sen=sen.replace("f**", 'fuck')
    sen=sen.replace(" fxk ", 'fuck')
    sen=sen.replace(" wil ", 'will')
    sen=sen.replace(" il ", 'i will')
    sen=sen.replace(" asss ", ' ass ')
    sen=sen.replace(" lifeles ", ' lifeless ')
    sen=sen.replace(" ugler ", ' uglier ')
    sen=sen.replace(" you're ", ' you are ') 
    sen=sen.replace(",", ' ') 
    sen=sen.replace("?", '.') 
    sen=sen.lower()
    for s in w:
        n=s+s+s
        sen=sen.replace(n, s)
    sen=" ".join(sen.split())
    QA.append(sen)
# print(QA)
q_a=QA
print(len(q_a))

9998


In [127]:
# my_dict = { 'q_a':q_a,'label':label}
# df = pd.DataFrame(my_dict)
# df.to_csv('formspring_all',index=False)

In [65]:
# Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=q_a
comments_attack['label']=label

In [66]:
dataframe = comments_attack
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))



In [67]:
print(dataframe['label'])

0       0
1       0
2       1
3       1
4       0
       ..
9993    0
9994    0
9995    0
9996    0
9997    0
Name: label, Length: 9998, dtype: int64


In [68]:
print(dataframe['comment'])

0       how to tell if a guy is gay if they seem super...
1                                          idk im not gay
2             this person is 100 fake so beware isabella 
3       oof lmao how is she fake whos this lmao shit f...
4       if your loved one left and cut all ties with y...
                              ...                        
9993                               basically anything omg
9994    why is it that when i ask someone so random as...
9995                   thats when my bitch mode comes out
9996           post a picture of your favorite celebrity 
9997                                     hunter hayes omg
Name: comment, Length: 9998, dtype: object


In [53]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.15, random_state=42)

In [54]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  8498
test comments length:  1500


In [56]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# word level ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)


In [57]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [58]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [59]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, WordLevel TF-IDF:   accuracy: 0.882     f1 score: 0.8304610797716455
NB, N-Gram Vectors:   accuracy: 0.886     f1 score: 0.8435533004221624
NB, CharLevel Vectors:   accuracy: 0.886   f1 score: 0.8445175094899428


In [60]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LR, WordLevel TF-IDF:   accuracy: 0.906   f1 score: 0.883751801347232
LR, N-Gram Vectors:   accuracy: 0.8846666666666667   f1 score: 0.836529911343034
LR, CharLevel Vectors:   accuracy: 0.9126666666666666   f1 score: 0.8935163965405424


In [61]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [62]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 3):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [63]:
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/3
8498/8498 [==============================] - 13s 2ms/step - loss: 0.4985 - accuracy: 0.8633
Epoch 2/3
8498/8498 [==============================] - 13s 2ms/step - loss: 0.3407 - accuracy: 0.8807
Epoch 3/3
8498/8498 [==============================] - 13s 1ms/step - loss: 0.2633 - accuracy: 0.8962
CNN, Word Embeddings acuuracy accuracy:0.903333306312561     f1 score: 0.8893836035059016
